In [ ]:
from jrl.robots import Panda
from jrl.evaluation import pose_errors_cm_deg
import torch

def assert_poses_almost_equal(poses_1, poses_2):
    pos_errors_cm, rot_errors_deg = pose_errors_cm_deg(poses_1, poses_2)
    assert (pos_errors_cm.max().item() < 0.01) and (rot_errors_deg.max().item() < 0.1)

robot = Panda()
joint_angles, poses = robot.sample_joint_angles_and_poses(n=5, return_torch=True) # sample 5 random joint angles and matching poses

# Run forward-kinematics
poses_fk = robot.forward_kinematics_batch(joint_angles) 
assert_poses_almost_equal(poses, poses_fk)

# Run inverse-kinematics
ik_sols = joint_angles + 0.1 * torch.randn_like(joint_angles) 
for i in range(5):
    ik_sols = robot.inverse_kinematics_single_step_levenburg_marquardt(poses, ik_sols)
assert_poses_almost_equal(poses, robot.forward_kinematics_batch(ik_sols))

In [ ]:
from utils.model import get_robot
robot = get_robot()

In [ ]:
from klampt.model import trajectory
milestones = [[0,0,0],[0.02,0,0],[1,0,0],[2,0,1],[2.2,0,1.5],[3,0,1],[4,0,-0.3]]
traj = trajectory.Trajectory(milestones=milestones)

In [ ]:
import numpy as np
endPoints = np.random.rand(2, 3)
traj = trajectory.Trajectory(milestones=endPoints)
endPoints

In [ ]:
from utils.settings import config as cfg

numSteps = 20
P_path = np.empty((numSteps, cfg.m))

for i in range(numSteps):
    iStep = i/numSteps
    point = traj.eval(iStep)
    print(f"{iStep}: {point}")
    P_path[i] = point

P_path

In [ ]:
mu = torch.zeros(size=(cfg.n,))
NUM_DATA = 100
NUM_SAMPLES = 1000

In [ ]:
test(robot=panda, P_ts=P_ts[:NUM_DATA], F=F, solver=flow, knn=knn, K=NUM_SAMPLES, print_report=True)

In [3]:
import numpy as np
np.isnan(np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [ ]:
from jrl.robots import Panda
from utils.settings import config as cfg
from utils.model import get_knn, get_flow_model
from utils.dataset import load_all_data, get_train_loader
robot = Panda()
J_tr, P_tr, P_ts, F = load_all_data(robot)
train_loader = get_train_loader(J=J_tr, P=P_tr, F=F, batch_size=128)
batch = next(iter(train_loader))
# knn = get_knn(P_tr=P_tr)

# # Build Generative model, NSF
# config = {
#         'subnet_width': 1024,
#         'subnet_num_layers': 3,
#         'num_transforms': 9,
#         'lr': 2.1e-4,
#         'lr_weight_decay': 2.7e-2,
#         'decay_step_size': 4e4,
#         'gamma': 5e-2,
#         'batch_size': 128,
#         'num_epochs': 10,
#     }
# # Neural spline flow (NSF) with 3 sample features and 5 context features
# solver, _, _ = get_flow_model(
#         enable_load_model=cfg.use_pretrained,
#         num_transforms=config["num_transforms"],
#         subnet_width=config["subnet_width"],
#         subnet_num_layers=config["subnet_num_layers"],
#         lr=config["lr"],
#         lr_weight_decay=config["lr_weight_decay"],
#         decay_step_size=config["decay_step_size"],
#         gamma=config["gamma"],
#         device='cuda')

In [ ]:
# import time 
# import torch
# import numpy as np
# from utils.utils import data_preprocess_for_inference
# K = 10
# position_errors = np.zeros(shape=(len(P_ts), K))
# orientation_errors = np.zeros(shape=(len(P_ts), K))


# # Data Preprocessing
# C = data_preprocess_for_inference(P=P_ts, F=F, knn=knn)


# time_begin = time.time()
# # Begin inference
# with torch.inference_mode():
#     J_hat = solver(C).sample((K,))
# J_hat = J_hat.detach().cpu().numpy()

# P_hat = robot.forward_kinematics_batch(torch.from_numpy(J_hat.reshape(-1, cfg.n)).to(cfg.device))
# P_hat = P_hat.reshape(J_hat.shape)

# P_ts_e = np.column_stack((P_ts, np.zeros(shape=(len(P_ts), 4))))
# P_ts_e.shape

# for i in range(len(J_hat)):
#     J_hat_i = torch.from_numpy(J_hat[i]).to(cfg.device)
#     results = solution_pose_errors(robot=robot, solutions=J_hat_i, target_poses=P_ts)
#     l2_errors, angular_errors = results
#     df = pd.DataFrame(l2_errors)
#     print(df.describe())

In [ ]:
# import numpy as np
# import pandas as pd
# import torch
# from jrl.evaluation import pose_errors, evaluate_solutions, solution_pose_errors
# from utils.utils import data_preprocess_for_inference


# def evaluate_solver(robot, solver, P_ts, F, knn, K=10):
#     C = data_preprocess_for_inference(P=P_ts, F=F, knn=knn)

#     with torch.inference_mode():
#         J_hat = solver(C).sample((K,))

#     l2_errs = np.empty((J_hat.shape[0], J_hat.shape[1]))
#     ang_errs = np.empty((J_hat.shape[0], J_hat.shape[1]))
#     if P_ts.shape[-1] == 3:
#         P_ts = np.column_stack((P_ts, np.ones(shape=(len(P_ts), 4))))
#     for i, J in enumerate(J_hat):
#         l2_errs[i], ang_errs[i] = solution_pose_errors(robot=robot, solutions=J, target_poses=P_ts)
        
#     l2_errs = l2_errs.flatten()
#     ang_errs = ang_errs.flatten()

#     # df = pd.DataFrame()
#     # df['l2_errs'] = l2_errs
#     # df['ang_errs'] = ang_errs
#     # print(df.describe())
#     return l2_errs.mean(), ang_errs.mean()


In [ ]:
from utils.utils import evaluate_solver
evaluate_solver(robot=robot, solver=solver, P_ts=P_ts, F=F, knn=knn, K=10)

In [ ]:
from jrl.robots import Panda
from jrl.evaluation import pose_errors_cm_deg
import torch

def assert_poses_almost_equal(poses_1, poses_2):
    pos_errors_cm, rot_errors_deg = pose_errors_cm_deg(poses_1, poses_2)
    assert (pos_errors_cm.max().item() < 0.01) and (rot_errors_deg.max().item() < 0.1)

robot = Panda()
joint_angles, poses = robot.sample_joint_angles_and_poses(n=5, return_torch=True) # sample 5 random joint angles and matching poses

# Run forward-kinematics
poses_fk = robot.forward_kinematics_batch(joint_angles) 
assert_poses_almost_equal(poses, poses_fk)

# Run inverse-kinematics
ik_sols = joint_angles + 0.1 * torch.randn_like(joint_angles) 
for i in range(5):
    ik_sols = robot.inverse_kinematics_single_step_levenburg_marquardt(poses, ik_sols)
assert_poses_almost_equal(poses, robot.forward_kinematics_batch(ik_sols))

In [ ]:
joint_angles

In [ ]:
df = pd.DataFrame(data=data, columns=[f'jt_{i}' for i in range(7)] + ['ml'])

In [ ]:
df.boxplot(by='ml', figsize=(10, 10), layout=(5, 2), fontsize=10)

In [ ]:
x_trans = load_numpy(file_path=config.x_trans_train_path)
data = np.column_stack((x_trans, label))
df_trans = pd.DataFrame(data=data, columns=[i for i in range(4)] + ['ml'])

In [ ]:
df_trans.boxplot(by='ml', figsize=(10, 10), layout=(5, 2), fontsize=10)

In [ ]:
for i in range(4):
    print(df_trans.groupby('ml')[i].describe())

In [ ]:
for i in range(7):
    print(df.groupby('ml')[f'jt_{i}'].describe())